In [ ]:
import pandas as pd
path = "/home/fiend/Documents"

In [ ]:
df = pd.read_csv(f'{path}/hc-transformed_df-2024-25.csv', parse_dates=True)

In [ ]:

merit_classification = {
    'judgement': [
        'Judgment Delivered- Case Closed',
        'Judgment Delivered',
        'Judgment Delivered- Acquittal',
        'Judgment Delivered- Convicted',
        'Grant Revoked',
        'Retrial'
        ],

    'ruling_delivered': [
        'Ruling Delivered- Case Closed', 
        'Ruling Delivered- Case Closed',

        ],
    'ruling_discharged': [
        'Ruling Delivered- Accused Discharged',
        ],
    'other_resolution': [
        'Grant Confirmed',
        'Limited Grant Issued',
        'Matter Withdrawn',
        'Application Withdrawn - Case Closed',
        'Consent Recorded - Case Closed',
        'Matter Settled Through Mediation',
        'Out Of Court Settlement Reached',
        'Dismissed For Want Of Prosecution - Case Closed',
        'Dismissed',
        'Appeal Dismissed',
        'Terminated',
        'Terminated/ Struck Out/ Dismissed/case Closed',
        'Struck Out',
        'Application Dismissed - Case Closed',
        'Application Allowed - Case Closed',
        'Matter Settled- Case Closed',
        'Ruling Delivered- Application Closed',
        'Consolidated- Case Closed',
        'Abated',
        'Placed In Probation',
        'Revision Declined',
        'Probation Orders Issued',
        'Appeal Rejected',
        'Interlocutory Judgement Entered',
    ],
    'ruling' : [
        'Ruling Delivered',
        'Ruling Delivered- Accused Put On Defense',
    ],
}



In [ ]:
# map merit_classification to a new column
df['merit_classification'] = df['outcome'].apply(
    lambda x: next((key for key, values in merit_classification.items() if x in values), None)
)

In [ ]:
df.groupby('merit_classification').size().reset_index(name='counts')

In [ ]:
judges = pd.read_csv(f'{path}/final_judges.csv')

In [ ]:
# filter df if judges in df['judges_1] 
df = df[df['judge_1'].isin(judges['name'])]


In [ ]:
df['activity_date'] = pd.to_datetime(df['activity_date'], format='mixed')

In [ ]:
# Extract the quarter from the activity_date column
df['quarter'] = df['activity_date'].dt.to_period('Q')

In [ ]:
df

In [ ]:
# 2024Q3 
quarter_1 = df[df['quarter'] == '2024Q3']
# Create a pivot table to summarize the data 
q1_judge_productivity = quarter_1.pivot_table(index='judge_1', columns='merit_classification', values='concluded', aggfunc='sum', fill_value=0).rename_axis(columns=None)

In [ ]:
# 2024Q3 
quarter_2 = df[df['quarter'] == '2024Q4']
# Create a pivot table to summarize the data 
q2_judge_productivity = quarter_2.pivot_table(index='judge_1', columns='merit_classification', values='concluded', aggfunc='sum', fill_value=0).rename_axis(columns=None)

In [ ]:
q1_judge_sittings = quarter_1.groupby('judge_1').size().sort_values(ascending=False)

In [ ]:
q2_judge_sittings = quarter_2.groupby('judge_1').size().sort_values(ascending=False)

In [ ]:
q1_judge_sittings.name = 'sittings'

q1_overall_judge_productivity = pd.merge(q1_judge_productivity, q1_judge_sittings, on='judge_1', how='outer')

In [ ]:
q2_judge_sittings.name = 'sittings'

q2_overall_judge_productivity = pd.merge(q2_judge_productivity, q2_judge_sittings, on='judge_1', how='outer')

In [ ]:
q2_overall_judge_productivity.to_csv(f'{path}/q2_overall_judge_productivity.csv', index=True)
q1_overall_judge_productivity.to_csv(f'{path}/q1_overall_judge_productivity.csv', index=True)

In [ ]:
df.pivot_table(index='judge_1', columns='merit_classification', values='concluded', aggfunc='sum', fill_value=0).rename_axis(columns=None).to_csv(f'{path}/merit_classification_by_judge.csv')

In [ ]:
df.pivot_table(index='court', columns='merit_classification', values='concluded', aggfunc='sum', fill_value=0).rename_axis(columns=None).to_csv(f'{path}/merit_classification_by_court.csv')

In [ ]:
df[df['resolution_mode'] == 'Ruling_Delivered'].groupby(['case_type','outcome'])['outcome'].count().sort_values(ascending=False)

In [ ]:
df[(df['court']=='Kisumu') & (df['concluded'] == 1)].groupby(['case_type','outcome'])['case_type'].count().sort_values(ascending=False)

In [ ]:
df

In [ ]:
# 1 - 4151
# 2 - 4603

In [ ]:
4151/57207

In [ ]:
4603/79869

In [ ]:
df.shape

In [ ]:
df.groupby('quarter')['adjourned'].count().reset_index(name='adjourned')

In [ ]:
period_start = pd.to_datetime('2023-07-01')
period_end = pd.to_datetime('2024-06-30')
# outcome = 'concluded'

# # Convert activity_date to datetime
df['activity_date'] = pd.to_datetime(df['activity_date'], errors='coerce')

kisumu_df = df[(df['court'] =='Kisumu') &
    (df['activity_date'] >= period_start) &
    (df['activity_date'] <= period_end)
]

In [ ]:
judges = ['Aburili, Roselyne', 'Saida,shariff Mwanaisha']
kisumu_df = kisumu_df[kisumu_df['judge_1'].isin(judges)].copy()

In [ ]:
kisumu_df = kisumu_df.copy(deep=True)

In [ ]:
kisumu_df.groupby('case_type')['concluded'].sum().reset_index(name='total').sort_values(by='total', ascending=False)

In [ ]:
kisumu_df.columns

In [ ]:
# determine Judicial Review cases resolved within 180 days
kisumu_df['concluded_within_180_days'] = kisumu_df['concluded'] & (kisumu_df['age'] <= 180)


In [ ]:
# percentage of judicial review cases resolved within 180 days
percentage_resolved_within_180_days = (
	kisumu_df[(kisumu_df['case_type'] == 'Judicial Review') & (kisumu_df['concluded_within_180_days'])]['concluded'].sum() /
	kisumu_df[kisumu_df['case_type'] == 'Judicial Review']['concluded'].sum()
) * 100


In [ ]:
kisumu_df[(kisumu_df['case_type'] == 'Judicial Review') & (kisumu_df['concluded_within_180_days'])]['concluded'].sum()

In [ ]:
kisumu_df[kisumu_df['case_type'] == 'Judicial Review']['concluded'].sum()

In [ ]:
percentage_resolved_within_180_days

In [ ]:
kisumu_df.loc[:, 'appearance_count'] = kisumu_df.groupby('case_number')['case_number'].transform('count')

In [ ]:
kisumu_df['resolved'] = kisumu_df.groupby('case_number')['concluded'].transform('max')


In [ ]:
kisumu_resolved = kisumu_df[kisumu_df['resolved'] >=1].copy()

In [ ]:
kisumu_resolved.loc[:, 'ruling_delivered'] = (kisumu_resolved['outcome'] == 'Ruling Delivered- Case Closed').astype(int)

In [ ]:
kisumu_resolved.loc[:, 'ruling_delivered_max'] = kisumu_resolved.groupby('case_number')['ruling_delivered'].transform('max')

In [ ]:
kisumu_ruling_df = kisumu_resolved[kisumu_resolved['ruling_delivered_max'] >= 1].copy()

In [ ]:
# ruling shceduled Ruling Date Given
kisumu_ruling_df.loc[:, 'ruling_scheduled'] = (kisumu_ruling_df['outcome'] == 'Ruling Date Given').astype(int)
kisumu_ruling_df.loc[:, 'ruling_scheduled_max'] = kisumu_ruling_df.groupby('case_number')['ruling_scheduled'].transform('max')

In [ ]:
ruling_not_scheduled = kisumu_ruling_df[kisumu_ruling_df['ruling_scheduled_max'] == 0]

In [ ]:
kisumu_df.query('case_number == "Kisumu/HCCRC/22/2020"')

In [ ]:
ruling_not_scheduled

In [ ]:
resolved_cases = kisumu_df.pivot_table(index='case_type', columns='productivity', values='concluded', aggfunc='sum', fill_value=0)

In [ ]:
resolved_cases_not_scheduled = ruling_not_scheduled.pivot_table(index='case_type', columns='productivity', values='concluded', aggfunc='sum', fill_value=0)

In [ ]:
# 
merged = pd.merge(resolved_cases, resolved_cases_not_scheduled, on='case_type', suffixes=('_resolved', '_not_scheduled'))

In [ ]:
merged

In [ ]:
ruling_not_scheduled.groupby('case_type')['ruling_delivered'].sum().reset_index(name='total').sort_values(by='total', ascending=False)

In [ ]:
ruling_not_scheduled[(ruling_not_scheduled['appearance_count'] >= 1) & (ruling_not_scheduled['appearance_count'] <= 2)].groupby('case_type')['ruling_delivered'].sum().reset_index(name='total').sort_values(by='total', ascending=False)

In [ ]:
kisumu_ruling_df.groupby('judge_1')['ruling_delivered'].sum().reset_index(name='total').sort_values(by='total', ascending=False)

In [ ]:
kisumu_ruling_df.groupby('case_type')['ruling_scheduled'].sum().reset_index(name='total').sort_values(by='total', ascending=False)